In [3]:
''' TODO: 
class segment: geometry/id, features, class

class segments: import, export, subset (leave-one-out, odd-one-out, ...)

'''
import ee
ee.Initialize()

segments = ee.FeatureCollection('ft:1Q7pTPj0E-Qp8xgED18uUI7hFFE9enHwMMUQJ0da0')

boundingBox = ee.Geometry.Polygon([[17.96, 59.36], [18.07, 59.42]])
point = ee.Geometry.Point([55, 25], 'EPSG:32633')

##----------PARAMETERS---------##
parameters_Stockholm = {
  'Area_Name': 'Stockholm',
  'Area': point,
  'CRS': 'EPSG:32633',
  'T0': '2017-06-01',
  'T1': '2017-09-30'
};

##projection scale for reducers 
##(Set >100 for testing, smaller <=30 for actual analysis)
proj_scale = 100;

S2_bands = ['B1','B2','B3','B4','B5','B6','B7','B8','B9','B10','B11','B12','NDVI','NDWI'];
S2_renamed_bands = ['S2_B1','S2_B2','S2_B3','S2_B4','S2_B5','S2_B6','S2_B7','S2_B8','S2_B9','S2_B10','S2_B11','S2_B12','S2_NDVI','S2_NDWI'];

L8_bands = ['B1','B2','B3','B4','B5','B6','B7','B10','B11','NDVI','NDWI'];
L8_renamed_bands = ['L8_B1','L8_B2','L8_B3','L8_B4','L8_B5','L8_B6','L8_B7','L8_B10','L8_B11','L8_NDVI','L8_NDWI'];

S1_bands = ['VV','VH','angle'];
S1_renamed_asc_bands = ['S1_Asc_VV','S1_Asc_VH','S1_Asc_angle']
S1_renamed_desc_bands = ['S1_Desc_VV','S1_Desc_VH','S1_Desc_angle']

##----------FUNCTIONS----------##
##------Landsat 8 Functions----##
## Function to cloud mask from the Fmask band of Landsat 8 SR data.
def maskL8sr(image):
  ##Bits 3 and 5 are cloud shadow and cloud, respectively.
  cloudShadowBitMask = ee.Number(2).pow(3).int();
  cloudsBitMask = ee.Number(2).pow(5).int();

  ## Get the pixel QA band.
  qa = image.select('pixel_qa');

  ## Both flags should be set to zero, indicating clear conditions.
  mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0).And(qa.bitwiseAnd(cloudsBitMask).eq(0));

  ## Return the masked image, scaled to [0, 1].
  return image.updateMask(mask).divide(10000);

##Function to add NDVI to Landsat 8
def addNdviL8(image):
    return image.addBands(image.normalizedDifference(['B5','B4']).rename('NDVI'))

##Function to add NDWI to Landsat 8
def addNdwiL8(image):
    return image.addBands(image.normalizedDifference(['B3','B5']).rename('NDWI'))



##-------Sentinel-2 Functions------##
##Function to mask clouds using the Sentinel-2 QA band
def maskS2clouds(image):
    qa = image.select('S2_QA60');
    
    ##Bits 10 and 11 are clouds and cirrus
    cloudBitMask = ee.Number(2).pow(10).int();
    cirrusBitMask = ee.Number(2).pow(11).int();
    
    ##Both flags should be set to zero, indicating clear conditions
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0));
    
    ##Return the masked and scaled data.
    return image.updateMask(mask).divide(10000);

##Function to add NDVI to Sentinel-2
def addNdviS2(image):
    return image.addBands(image.normalizedDifference(['B8', 'SB4']).rename('NDVI'));

##Function to add NDWI to Sentinel-2
def addNdwiS2(image):
    return image.addBands(image.normalizedDifference(['B3', 'B8']).rename('NDWI'));

##------Sentinel-1 Functions-----##
##Function to mask out pixels that did not have observations
def maskEmptyPixelsS1(image):
    ##Find pixels that had observations
    return image.updateMask(image.select('VV').gt(-25));
    

##---------MAIN----------##
parameters = parameters_Stockholm;
print(parameters['T0'])

##---Collect Sentinel-2 imagery----##
S2_Img = ee.ImageCollection('COPERNICUS/S2'
    ).filterDate(parameters['T0'], parameters['T1']
    ).filterBounds(parameters['Area']
    ).filterMetadata('CLOUDY_PIXE_PERCENTAGE','less_than', 20
    ).map(maskS2clouds
    ).map(addNdviS2
    ).map(addNdwiS2
    ).select(S2_bands, S2_renamed_bands  
    ).median();


##---Collect Sentinel-1 imagery----##
##---Descending---##
S1_Desc = ee.ImageCollection('COPERNICUS/S1_GRD'
    ).filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING')
    ).filterDate(parameters['T0'], parameters['T1']
    ).filterBounds(parameters['Area']
    ).map(maskEmptyPixelsS1
    ).select(S1_bands, S1_renamed_desc_bands);

S1_Desc_Mean = S1_Desc.mean();
S1_Desc_StdDev = ee.Image(S1_Desc.reduce(ee.Reducer.stdDev()).toFloat());

##---Ascending----##
S1_Asc = ee.ImageCollection('COPERNICUS/S1_GRD'
    ).filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING')
    ).filterDate(parameters['T0'], parameters['T1']
    ).filterBounds(parameters['Area']
    ).map(maskEmptyPixelsS1
    ).select(S1_bands, S1_renamed_asc_bands);

S1_Asc_Mean = S1_Asc.mean();
S1_Asc_StdDev = ee.Image(S1_Asc.reduce(ee.Reducer.stdDev()).toFloat());

print('Image collection:', S1_Asc.getInfo())
print('single Image:',S1_Asc_Mean.getInfo())

print

##---Collect Landsat 8 imagery----##
L8_Img = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR'
    ).filterDate(parameters['T0'], parameters['T1']
    ).filterBounds(parameters['Area']
    ).median();

 
##---Construct features for segments---##
##combined reducer for mean and standard deviation
Mean_StdDev_Reducer = ee.Reducer.mean(
    ).combine(ee.Reducer.stdDev(),'',True)

##add features to the segments (Feature Collection)
##segments = S2_Img.reduceRegions(segments,Mean_StdDev_Reducer,proj_scale, parameters['CRS'])

segments = S1_Asc_StdDev.reduceRegions(segments,Mean_StdDev_Reducer,proj_scale,parameters['CRS'])

print(segments.first().getInfo())

##segments = S1_Asc_StdDev.reduceRegions(segments,Mean_StdDev_Reducer,proj_scale)

##segments = S1_Desc_Mean.reduceRegions(segments,Mean_StdDev_Reducer,proj_scale)

##segments = S1_Desc_StdDev.reduceRegions(segments,Mean_StdDev_Reducer,proj_scale)

##segments = L8_Img.reduceRegions(segments,Mean_StdDev_Reducer,proj_scale)

##print(segments.first().getInfo())

##Compute statistics over bounding box
##Add reducer output to the Features in the collection.
##stats = L8_Img.reduceRegions(segments, ee.Reducer.mean(), 30);
##foo = stats.first().get('id').getInfo()

##print(foo)

##mean = L8_Img.reduceRegion(ee.Reducer.mean(), boundingBox, 10000);

##statsDict = stats.toList()

##print(statsDict)






2017-06-01
Image collection: {'type': 'ImageCollection', 'bands': [], 'id': 'COPERNICUS/S1_GRD', 'version': 1518038988511568, 'properties': {'date_range': [1412294400000.0, 1517788800000.0], 'period': 0.0, 'system:visualization_0_min': [-25.0], 'system:visualization_0_bands': ['HH'], 'thumb': 'https://mw1.google.com/ges/dd/images/s1_thumb.png', 'description': '<p>The Sentinel-1 mission provides data from a dual-polarization\nC-band Synthetic Aperture Radar (SAR) instrument.  This collection\nincludes the S1 Ground Range Detected (GRD) scenes, processed\nusing the Sentinel-1 Toolbox to generate a calibrated, ortho-corrected\nproduct. The collection is updated weekly.</p>\n<p>This collection\ncontains all of the GRD scenes.  Each scene has one of 3 resolutions\n(10, 25 or 40 meters), 4 band combinations (corresponding to\nscene polarization) and 3 instrument modes.  Use of the collection\nin a mosaic context will likely require filtering down to a homogenous\nset of bands and parameters.

single Image: {'type': 'Image', 'bands': []}


EEException: Image.reduceRegions: Need 1 bands for <Reducer>, image has 0.